<a href="https://colab.research.google.com/github/GenAIUnplugged/langchain_series/blob/main/Gradio_UI_with_Conversational_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# Install necessary packages if not already
!pip install langchain faiss-cpu openai tiktoken gradio langchain-core langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [10]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

In [11]:
# 1. Import required modules
import gradio as gr
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import Document

In [16]:
llm = ChatOpenAI(model="gpt-4o-mini",temperature=0)

<ipython-input-16-2284586df914>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o-mini",temperature=0)


In [17]:
# 2. Sample documents
docs = [
    Document(page_content="LangChain is a framework for building LLM-powered applications."),
    Document(page_content="FAISS is a vector store used for similarity search, developed by Facebook."),
    Document(page_content="ChatGPT is a language model created by OpenAI, capable of generating human-like text.")
]

In [18]:
# 3. Create embeddings and FAISS index
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding=embeddings)
retriever = vectorstore.as_retriever()

In [19]:
# 4. Create memory for conversation
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [20]:
# 6. Create the Conversational Retrieval Chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory
)

In [21]:
# 7. Define Gradio interface
def chatbot(input_text, chat_history):
    # Update the conversation history
    query = input_text
    response = qa_chain.run(query)
    chat_history.append((query, response))
    return chat_history, chat_history

In [22]:
# 8. Setup Gradio interface
iface = gr.Interface(
    fn=chatbot,
    inputs=[
        gr.Textbox(label="Ask a question", placeholder="Type your question here..."),
        gr.Chatbot(label="Chat History")
    ],
    outputs=[
        gr.Chatbot(label="Chat History"),
        gr.Chatbot(label="Chat History (for history)")
    ],
    live=True,
    title="Conversational RAG Chatbot",
    description="Ask questions and the bot will respond based on the document context and conversation history."
)

<ipython-input-22-1393836845fd>:6: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  gr.Chatbot(label="Chat History")
<ipython-input-22-1393836845fd>:9: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  gr.Chatbot(label="Chat History"),
<ipython-input-22-1393836845fd>:10: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set ty

In [23]:
# 9. Launch the interface
iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://af03a57317e4db9fd2.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
